<a href="https://colab.research.google.com/github/Joacoromero06/Proyects/blob/main/grafosNoDirigidos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from collections import defaultdict
class Graph:
  def __init__(self):
    self.adj=defaultdict(list)
  def add_edge(self,u,v,weight):
    self.adj[u].append((v,weight))
    self.adj[v].append((u,weight))
  def vertexs(self):
    return list(self.adj.keys())
  def neighbours(self,u):
    return self.adj[u]
  def __str__(self):
    return str(self.adj)

In [ ]:
import heapq
def prim(graph,start):
  mst_edges=[]
  U_visited=set()
  min_heap=[(0,start,None)]
  while min_heap:
    weight,vertex,u=heapq.heappop(min_heap)
    if vertex in U_visited:
      continue
    U_visited.add(vertex)
    if u is not None:
      mst_edges.append((u,vertex,weight))
    for v,w in graph.neighbours(vertex):
      if v not in U_visited:
        heapq.heappush(min_heap,(w,v,vertex))

  return mst_edges

In [ ]:
# Crear grafo
g = Graph()
g.add_edge(1, 2, 2)
g.add_edge(1, 4, 6)
g.add_edge(2, 3, 3)
g.add_edge(2, 5, 5)
g.add_edge(3, 5, 7)
g.add_edge(4, 5, 9)

# Ejecutar Prim
mst = prim(g, start=1)
print("Aristas del MST:", mst)


Aristas del MST: [(1, 2, 2), (2, 3, 3), (2, 5, 5), (1, 4, 6)]


In [ ]:
class DSU:
  def __init__(self,n):
    self.parent=[i for i in range(n)]
    self.rank=[0]*n
  def find(self,x):
    if self.parent[x]!=x:
      self.parent[x]=self.find(self.parent[x])
    return self.parent[x]
  def union(self,x,y):
    rx,ry=self.find(x),self.find(y)
    if rx==ry:
      return False
    if self.rank[rx]<self.rank[ry]:
      self.parent[rx]=ry
    elif self.rank[rx]>self.rank[ry]:
      self.parent[ry]=rx
    else:
      self.parent[ry]=rx
      self.rank[rx]+=1
    return True

In [ ]:
def kruskal(n,aristas):
  aristas.sort()
  dsu=DSU(n)
  mst=[]
  total_cost=0
  for w,u,v in aristas:
    if dsu.union(u,v):
      mst.append((w,u,v))
      total_cost+=w
  return mst,total_cost

In [ ]:
# Grafo con 5 vértices
edges = [
    (2, 0, 1),
    (6, 0, 3),
    (3, 1, 2),
    (8, 1, 3),
    (5, 1, 4),
    (7, 2, 4),
    (9, 3, 4)
]

mst, cost = kruskal(5, edges)
print("MST:", mst)
print("Costo total:", cost)


MST: [(2, 0, 1), (3, 1, 2), (5, 1, 4), (6, 0, 3)]
Costo total: 16


In [ ]:
from collections import defaultdict
from typing import Dict, List, Set, Tuple

class ArticulationPoints:
    def __init__(self, num_vertices: int):
        self.V = num_vertices
        self.graph: Dict[int, List[int]] = defaultdict(list)
        self.time = 0
        self.num: List[int] = [-1] * self.V   # orden de descubrimiento
        self.low: List[int] = [-1] * self.V   # valor low
        self.parent: List[int] = [-1] * self.V
        self.ap: Set[int] = set()  # puntos de articulación encontrados

    def add_edge(self, u: int, v: int):
        """Agrega una arista no dirigida"""
        self.graph[u].append(v)
        self.graph[v].append(u)

    def _dfs(self, u: int):
        """DFS modificada para calcular low[] y encontrar puntos de articulación"""
        self.time += 1
        self.num[u] = self.low[u] = self.time
        children = 0  # número de hijos en el árbol DFS

        for v in self.graph[u]:
            if self.num[v] == -1:
                # v no visitado → arista de árbol
                self.parent[v] = u
                children += 1
                self._dfs(v)

                # Al volver, actualizamos low[u]
                self.low[u] = min(self.low[u], self.low[v])

                # Caso 1: u es raíz y tiene más de un hijo
                if self.parent[u] == -1 and children > 1:
                    self.ap.add(u)

                # Caso 2: u no es raíz y low[v] >= num[u]
                if self.parent[u] != -1 and self.low[v] >= self.num[u]:
                    self.ap.add(u)

            elif v != self.parent[u]:
                # Arista de retroceso
                self.low[u] = min(self.low[u], self.num[v])

    def find_articulation_points(self) -> Set[int]:
        """Encuentra todos los puntos de articulación"""
        for u in range(self.V):
            if self.num[u] == -1:
                self._dfs(u)
        return self.ap


# Ejemplo de uso
if __name__ == "__main__":
    g = ArticulationPoints(5)
    edges = [(0, 1), (1, 2), (2, 0), (1, 3), (3, 4)]
    for u, v in edges:
        g.add_edge(u, v)

    puntos = g.find_articulation_points()
    print("Puntos de articulación:", puntos)  # Output esperado: {1, 3}


In [1]:
from collections import deque

INF = float('inf')

class HopcroftKarp:
    def __init__(self, graph, U, V):
        """
        graph: diccionario {u: [v1, v2, ...]} con aristas desde U a V
        U: conjunto de vértices del lado izquierdo
        V: conjunto de vértices del lado derecho
        """
        self.graph = graph
        self.U = U
        self.V = V
        self.pair_u = {u: None for u in U}  # emparejamiento en U
        self.pair_v = {v: None for v in V}  # emparejamiento en V
        self.dist = {}

    def bfs(self):
        """Construye niveles y devuelve True si hay al menos un camino aumentado."""
        queue = deque()
        for u in self.U:
            if self.pair_u[u] is None:  # vértice libre
                self.dist[u] = 0
                queue.append(u)
            else:
                self.dist[u] = INF

        found = False
        while queue:
            u = queue.popleft()
            for v in self.graph.get(u, []):
                u2 = self.pair_v[v]
                if u2 is None:
                    found = True  # encontramos un vértice libre en V
                elif self.dist[u2] == INF:
                    self.dist[u2] = self.dist[u] + 1
                    queue.append(u2)
        return found

    def dfs(self, u):
        """Intenta encontrar un camino aumentado desde u."""
        for v in self.graph.get(u, []):
            u2 = self.pair_v[v]
            if u2 is None or (self.dist[u2] == self.dist[u] + 1 and self.dfs(u2)):
                self.pair_u[u] = v
                self.pair_v[v] = u
                return True
        self.dist[u] = INF
        return False

    def max_matching(self):
        matching = 0
        while self.bfs():
            for u in self.U:
                if self.pair_u[u] is None:
                    if self.dfs(u):
                        matching += 1
        return matching


In [2]:
U = {"u1", "u2", "u3"}
V = {"v1", "v2", "v3"}
graph = {
    "u1": ["v1", "v2"],
    "u2": ["v1"],
    "u3": ["v2", "v3"]
}

hk = HopcroftKarp(graph, U, V)
max_match = hk.max_matching()
print("Emparejamiento máximo:", max_match)
print("pair_u:", hk.pair_u)
print("pair_v:", hk.pair_v)


Emparejamiento máximo: 3
pair_u: {'u3': 'v3', 'u2': 'v1', 'u1': 'v2'}
pair_v: {'v3': 'u3', 'v2': 'u1', 'v1': 'u2'}
